## 계산 그래프
계산 그래프로 문제를 푸는 이유는 그냥 단순하게 보여주기 위해서 풀음 ㅇㅅ<

## 연쇄 법칙(Chain Rule)

https://ko.wikipedia.org/wiki/%EC%97%B0%EC%87%84_%EB%B2%95%EC%B9%99

참고하시길

### 연쇄 법칙의 정의

합성 함수의 미분에 대한 성질
> 합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.

## 역전파

### 덧셈 노드의 역전파

단순하게 입력 신호를 다음 노드로 출력할 뿐이므로 그대로를 다음 노드로 전달

### 곱셈 노드의 역전파

상류의 값에 순전파 때의 입력 신호들을 _서로 바꾼 값_을 곱해서 하류로 보냄
서로 바꾼 값이란 순전파 때 x였다면, 역전파에서는 y, 순전파 때 y였다면 역전파에서는 x로 바꾼다는 의미

## 단순한 계층 구현하기

### 곱셈 계층


In [1]:
class MulLayer:
    def __init__(self):
        # 초기화, 순전파 시의 입력값을 유지하기 위해 사용
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        # x, y를 인수로 받고 두 값을 곱해서 반환
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        # 상류에서 넘어온 미분(dout)에 순전파 때의 값을 '서로 바꿔 곱한 후' 하류로 흘림
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

** 실질 구현 **

In [4]:
apple = 100
apple_num = 2
tax = 1.1

# Layers
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print("tax를 제외하면: ", apple_price)
print("tax를 포함하면: ", price)

tax를 제외하면:  200
tax를 포함하면:  220.00000000000003


** 역전파 **

In [5]:
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


### 덧셈 계층

In [6]:
class AddLayer:
    def __init__(self):
        # 덧셈 계층에서는 초기화가 필요 없으므로 __init__()은 아무 일도 하지 않음
        pass
    
    def forward(self, x, y):
        # 단순하게 두 숫자를 더해서 반환
        out = x + y
        return out
    
    def backward(self, dout):
        # 상류층에서 내려온 미분(dout)을 그대로 하류로 흘림
        dx = dout * 1
        dy = dout * 1
        return dx, dy

**실질 구현**

In [9]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# Layers
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num) # (1)
orange_price = mul_orange_layer.forward(orange, orange_num) # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price) # (3)
price = mul_tax_layer.forward(all_price, tax) # (4)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice) # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price) # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price) # (2)
daplle, dapple_num = mul_apple_layer.backward(dapple_price) # (1)

print(price)
print(dapple_num, dapple, dorange, dorange_num)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0


## 활성화 함수 계층 구현하기

** ReLU 계층 **
- 순전파 때의 입력인 x가 0보다 크면 상류의 값을 그대로 하류로 흘림
- 순전파 때 x가 0 이하이면 역전파 때는 하류로 신호를 보내지 않음

In [1]:
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

Instance Variable mask: True/False, Numpy array

In [3]:
import numpy as np

x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)
mask = ( x <= 0 )
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


** Sigmoid 계층 **
- 1단계: 상류의 예측값에 -y^2을 곱해서 하류로 전달
- 2단계: 상류의 값을 여과없이 하류로 내보냄
- 3단계: exp(-x)를 곱해서 하류로 전파
- 4단계: 순전파 때의 값을 '서로 바꿔' 곱

In [4]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

## Affine/Softmax 계층 구현하

### Affine 계층
- 신경망의 순전파에서는 가중치 신호의 총합을 계산하기 때문에, 행렬의 내적(_np.dot()_)을 사용
- 여태까지의 계산 그래프에서는 노드 사이에 '스칼라값'이 흘렀지만, '행렬'을 흐르게 해야 한다.
- 행렬의 내적에서는 대응하는 차원의 원소 수를 일치시켜야 한다.

** 배치용 Affine 계층 **
- 데이터 N개를 묶어 순전파하는 경우
- 편향에 주의해야하는데, 편향의 역전파는 그 두 데이터에 대한 미분을 데이터마다 더해서 구

In [5]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

** Softmax-with-Loss 계층 **
- 손실 함수인 교차 엔트로피 오차도 포함하여 구현
- 신경망의 출력과 정답 레이블의 오차를 효율적으로 앞 계층에 전달하여<br>신경망의 출력(Softmax의 출력)이 정답 레이블과 가까워지도록 가중치 매개변수의 값을 조정해야 함

In [6]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실
        self.y = None # 소프트 맥스의 출력
        self.t = None # 정답 레이블
        
    def foward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout = 1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

### 오차역전파법 구현하기

** 신경망 학습의 전체 그림 **<br>
- 전체: 신경망에 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'
 - 1단계: 미니배치 - 훈련 데이터 중 일부를 무작위로 가져온다. 이 미니배치의 손실 함수 값을 줄이는 것을 목표로 함
 - 2단계: 기울기 산출 - 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구해서, 손실 함수의 값을 가장 작게 하는 방향 제시
 - 3단계:  매개변수 갱신- 가중치 매개변수를 기울기 방향으로 아주 조금 갱신
 - 4단계: 반복

** 오차역전파법을 적용한 신경망 구현하기 **<br>
- 2층 신경망 TwoLayerNet 구현

In [ ]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
# 이거 인덴트가 이상한데    